In [3]:
import import_ipynb

from category_list_clean import category_clean_data

importing Jupyter notebook from category_list_clean.ipynb


In [4]:
#Calling code to retrieve the cleaned category list
category_list = category_clean_data()

category_list

,Category_id,Category_Name
0,1,Film & Animation
1,2,Autos & Vehicles
2,10,Music
3,15,Pets & Animals
4,17,Sports
5,18,Short Movies
6,19,Travel & Events
7,20,Gaming
8,21,Videoblogging
9,22,People & Blogs
